# Many to many

In [ ]:
from sqlalchemy.orm import DeclarativeBase # Декларативный, базовый класс
from sqlalchemy.orm import mapped_column, declared_attr
from sqlalchemy.orm import Session # для создания записей в бд
from sqlalchemy.orm import Mapped
from sqlalchemy import ForeignKey, String
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine, text, select
from typing import List
import sqlite3
from sqlalchemy import func
from sqlalchemy import Column, Table
from datetime import datetime

Tutorial from SQLAlchemy

In [ ]:
class Base(DeclarativeBase):
  __abstract__ = True # модель не должна быть создана в базе данных

  @declared_attr # задаем имя таблицы по умолчанию
  def __tablename__(cls) -> str:
    return f"{cls.__name__.lower()}s"

  id: Mapped[int] = mapped_column(primary_key=True)


class Order(Base):
  __tablename__ = "orders"

  promocode: Mapped[str | None]
  created_at: Mapped[datetime] = mapped_column(
      server_default=func.now(), # что будет на стороне сервера при создании это записи
      default=datetime.utcnow # что будет по умолчанию
    )
  products: Mapped[list["Product"]] = relationship(secondary="order_product_associstion",
                                                   back_populates="orders") # ссылаемся на промежуточную таблицу


order_product_associstion_table = Table(
    "order_product_associstion",  # имя таблицы
    Base.metadata, # мета-данные
    # колонки в любом количестве (какая таблица, а какая справа - неважно)
    # Здесь мы создаем колонки
    Column("order_id", ForeignKey("orders.id"), primary_key=True), # primary_key - один товар в корзине может быть только один раз
    Column("product_id", ForeignKey("products.id"), primary_key=True)
)


class Product(Base):
  __tablename__ = "products"

  name: Mapped[str]
  description: Mapped[str]
  price: Mapped[int]

  orders: Mapped[list["Product"]] = relationship(secondary="order_product_associstion",
                                                   back_populates="products") # ссылаемся на промежуточную таблицу